In [16]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np
from keras.models import Model, model_from_json
from keras.layers import Dense, GlobalAveragePooling2D, Input, Rescaling
from keras.utils import image_dataset_from_directory

In [6]:
img_width, img_height = 224, 224
epochs = 2
batch_size = 32

In [7]:
inputs = Input(shape=(img_width, img_height, 3))
x = Rescaling(1./255)(inputs) 
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=x)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
train_dataset = image_dataset_from_directory(
    'data',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.1,
    subset='training',
    seed=42,
    label_mode="categorical"
)

validation_dataset = image_dataset_from_directory(
    'data',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.1,
    subset='validation',
    seed=42,
    label_mode="categorical"
)

Found 1050 files belonging to 3 classes.
Using 945 files for training.
Found 1050 files belonging to 3 classes.
Using 105 files for validation.


In [10]:
model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=validation_dataset,
    verbose=1
)

Epoch 1/2
30/30 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.7989 - loss: 0.6165 - val_accuracy: 0.9810 - val_loss: 0.2216
Epoch 2/2
30/30 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.9672 - loss: 0.1763 - val_accuracy: 0.9905 - val_loss: 0.0902


In [11]:
loss, accuracy = model.evaluate(validation_dataset)
print(f"Validation loss: {loss:.4f}")
print(f"Validation accuracy: {accuracy:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.9905 - loss: 0.0902
Validation loss: 0.0902
Validation accuracy: 0.9905


In [12]:
classes = ['Chulpi_Cancha', 'Indurata', 'Rugosa']

In [13]:
model_json = model.to_json()
json_file = open("vgg16_corn.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("vgg16_corn.weights.h5")
json_file = open("vgg16_corn.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("vgg16_corn.weights.h5")
loaded_model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

In [15]:
img = image.load_img('tester2.png', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
prediction = loaded_model.predict(x)
print(prediction)
print(classes[np.argmax(prediction)])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
[[0.15822157 0.7721857  0.06959278]]
Indurata


In [ ]:
##https://commons.wikimedia.org/wiki/File:LPCC-788-Grans_de_Zea_mays.jpg